# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [41]:
from pyspark.sql import SparkSession

In [42]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [43]:
data = spark.read.csv('customer_churn.csv', header=True, inferSchema=True)

In [44]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [45]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [46]:
data.show(2)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|   Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|Wilson PLC|    1|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+
only showing top 2 rows



In [47]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Format for MLlib

We'll ues the numerical columns. We'll include Account Manager because its easy enough, but keep in mind it probably won't be any sort of a signal because the agency mentioned its randomly assigned!

In [48]:
from pyspark.ml.feature import VectorAssembler

In [49]:
assembler = VectorAssembler(inputCols=[ 'Age','Account_Manager','Years','Num_Sites','Total_Purchase'],
                            outputCol='features')

In [50]:
output = assembler.transform(data)

In [51]:
output.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- features: vector (nullable = true)



In [52]:
output.select(['features','churn']).show(4)

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,0.0,7.22,8....|    1|
|[41.0,0.0,6.5,11....|    1|
|[38.0,0.0,6.67,12...|    1|
|[42.0,0.0,6.71,10...|    1|
+--------------------+-----+
only showing top 4 rows



In [53]:
final_data = output.select(['features','churn'])

Split Data: Train and Test

In [54]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [55]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [56]:
lg = LogisticRegression(featuresCol='features', labelCol='churn', predictionCol='prediction')

In [57]:
fitted_lg_model = lg.fit(train_data)

In [58]:
training_summary = fitted_lg_model.summary

In [59]:
training_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                635|                635|
|   mean|0.16850393700787403|0.12913385826771653|
| stddev|0.37460826803441455| 0.3356124013597334|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



### Evaluate results

Let's evaluate the results on the data set we were given (using the test data)

In [60]:
predictions_and_labels = fitted_lg_model.evaluate(test_data)

In [61]:
predictions_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[27.0,1.0,5.3,7.0...|    0|[5.20942395296651...|[0.99456488117535...|       0.0|
|[28.0,1.0,5.74,10...|    0|[1.53928779261837...|[0.82336116737410...|       0.0|
|[29.0,1.0,5.82,8....|    0|[3.72075586830712...|[0.97635687668781...|       0.0|
|[30.0,0.0,5.56,9....|    0|[2.93267392499818...|[0.94943819204180...|       0.0|
|[30.0,1.0,4.13,7....|    0|[5.65039927159735...|[0.99649620711962...|       0.0|
|[30.0,1.0,5.22,8....|    1|[4.01563608362149...|[0.98228789413475...|       0.0|
|[30.0,1.0,5.96,9....|    0|[2.47567743706810...|[0.92241903018030...|       0.0|
|[30.0,1.0,7.31,7....|    0|[3.95389259039149...|[0.98118104874752...|       0.0|
|[31.0,0.0,3.58,7....|    0|[6.18567664168498...|[0.99794552149793...|       0.0|
|[31.0,1.0,4.52,

### Using AUC

In [62]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [63]:
auc = churn_eval.evaluate(predictions_and_labels.predictions)

In [64]:
auc

0.7678085061805993

[Common question - what is a good AUC value?](https://stats.stackexchange.com/questions/113326/what-is-a-good-auc-for-a-precision-recall-curve)

### Predict on brand new unlabeled data

need to evaluate the new_customers.csv file!

In [65]:
final_lg_model = lg.fit(final_data)

In [66]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [67]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [69]:
test_new_customers = assembler.transform(new_customers)

In [71]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [70]:
test_new_customers.show(3)

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,1.0,7.71,8....|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,1.0,9.28,15...|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,1.0,1.0,15....|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+---

In [72]:
final_results = final_lg_model.transform(test_new_customers)

In [74]:
final_results.select(['Company', 'prediction']).show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



That is it! Now we know that we should assign Acocunt Managers to Cannon-Benson,Barron-Robertson,Sexton-GOlden, and Parks-Robbins!